<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Dense_BIC_9_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from tabulate import tabulate

# Bayesian Network Data Generation 500, ..., 20000 Samples (dense)

In [3]:
# Function to generate CPDs for the expanded structure with 9 nodes
def generate_cpds():
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random probabilities for IRT given IR
    irt_given_ir_probs = np.random.rand(3, 3)
    irt_given_ir_probs /= irt_given_ir_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for EI given IR and IRT
    ei_given_ir_irt_probs = np.random.rand(3, 3, 3)
    ei_given_ir_irt_probs /= ei_given_ir_irt_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for GEO
    geo_probs = np.random.rand(3)
    geo_probs /= geo_probs.sum()

    # Generate random probabilities for UE given GEO
    ue_given_geo_probs = np.random.rand(3, 3)
    ue_given_geo_probs /= ue_given_geo_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for INF (Inflation Rate)
    inf_probs = np.random.rand(3)
    inf_probs /= inf_probs.sum()

    # Generate random probabilities for GDP given INF
    gdp_given_inf_probs = np.random.rand(3, 3)
    gdp_given_inf_probs /= gdp_given_inf_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for CP (Consumer Price Index) given GDP
    cp_given_gdp_probs = np.random.rand(3, 3)
    cp_given_gdp_probs /= cp_given_gdp_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for SP given IR, EI, and UE
    sp_probs = np.random.rand(3, 3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return (
        ir_probs, irt_given_ir_probs, ei_given_ir_irt_probs, geo_probs, ue_given_geo_probs,
        inf_probs, gdp_given_inf_probs, cp_given_gdp_probs, sp_probs
    )

# Function to generate and save samples with the expanded structure
def generate_and_save_samples(ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, inf_probs, gdp_probs, cp_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample each node based on the generated probabilities
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        irt_probs_given_ir = irt_probs[:, ir_state_idx]
        irt_state_idx = np.random.choice(3, p=irt_probs_given_ir)
        irt_state = ['decreasing', 'steady', 'increasing'][irt_state_idx]
        irt_prob = irt_probs_given_ir[irt_state_idx]

        ei_probs_given_ir_irt = ei_probs[:, ir_state_idx, irt_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir_irt)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs_given_ir_irt[ei_state_idx]

        geo_state_idx = np.random.choice(3, p=geo_probs)
        geo_state = ['recession', 'stable', 'growth'][geo_state_idx]
        geo_prob = geo_probs[geo_state_idx]

        ue_probs_given_geo = ue_probs[:, geo_state_idx]
        ue_state_idx = np.random.choice(3, p=ue_probs_given_geo)
        ue_state = ['high', 'medium', 'low'][ue_state_idx]
        ue_prob = ue_probs_given_geo[ue_state_idx]

        inf_state_idx = np.random.choice(3, p=inf_probs)
        inf_state = ['low', 'medium', 'high'][inf_state_idx]
        inf_prob = inf_probs[inf_state_idx]

        gdp_probs_given_inf = gdp_probs[:, inf_state_idx]
        gdp_state_idx = np.random.choice(3, p=gdp_probs_given_inf)
        gdp_state = ['decline', 'stable', 'growth'][gdp_state_idx]
        gdp_prob = gdp_probs_given_inf[gdp_state_idx]

        cp_probs_given_gdp = cp_probs[:, gdp_state_idx]
        cp_state_idx = np.random.choice(3, p=cp_probs_given_gdp)
        cp_state = ['decrease', 'stable', 'increase'][cp_state_idx]
        cp_prob = cp_probs_given_gdp[cp_state_idx]

        sp_probs_given_ir_ei_ue = sp_probs[:, ir_state_idx, ei_state_idx, ue_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei_ue)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
        sp_prob = sp_probs_given_ir_ei_ue[sp_state_idx]

        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'IRT_State': irt_state,
            'IRT_Prob': f'{irt_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'GEO_State': geo_state,
            'GEO_Prob': f'{geo_prob:.4f}',
            'UE_State': ue_state,
            'UE_Prob': f'{ue_prob:.4f}',
            'INF_State': inf_state,
            'INF_Prob': f'{inf_prob:.4f}',
            'GDP_State': gdp_state,
            'GDP_Prob': f'{gdp_prob:.4f}',
            'CP_State': cp_state,
            'CP_Prob': f'{cp_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_ir_ei_ue]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}'
        })

    output_df = pd.DataFrame(output_data)
    output_df.to_csv(filename, index=False)
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

for size in sample_sizes:
    # Generate the CPDs with the expanded structure
    ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, inf_probs, gdp_probs, cp_probs, sp_probs = generate_cpds()
    generate_and_save_samples(ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, inf_probs, gdp_probs, cp_probs, sp_probs, size, f'combined_probabilities_{size}.csv')

print("\nGeneration and saving of individual samples complete for all sample sizes!")


Sample size: 50 - First few rows of generated samples:

+----+------------+-----------+-------------+------------+------------+-----------+-------------+------------+------------+-----------+-------------+------------+-------------+------------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|    | IR_State   |   IR_Prob | IRT_State   |   IRT_Prob | EI_State   |   EI_Prob | GEO_State   |   GEO_Prob | UE_State   |   UE_Prob | INF_State   |   INF_Prob | GDP_State   |   GDP_Prob | CP_State   |   CP_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability |
+====+============+===========+=============+============+============+===========+=============+============+============+===========+=============+============+=============+============+============+===========+=================================================+===================+=========================+
|  0 | med

# LBN Dense BIC & Entropy

In [4]:
# Sample sizes to loop through
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000,
                2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000,
                18000, 19000, 20000]

# Initialize list to store K-L divergence and standard deviation values for each sample size
results = []

# Small smoothing value to avoid zero probabilities
epsilon = 1e-10

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dataset for the current sample size
    data_file = f'combined_probabilities_{sample_size}.csv'
    df = pd.read_csv(data_file)

    # Manually encode categorical variables for IR, IRT, EI, GEO, UE, INF, GDP, CP, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    irt_map = {'decreasing': 0, 'steady': 1, 'increasing': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    geo_map = {'recession': 0, 'stable': 1, 'growth': 2}
    ue_map = {'high': 0, 'medium': 1, 'low': 2}
    inf_map = {'low': 0, 'medium': 1, 'high': 2}
    gdp_map = {'decline': 0, 'stable': 1, 'growth': 2}
    cp_map = {'decrease': 0, 'stable': 1, 'increase': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR_State'].map(ir_map)
    df['IRT_encoded'] = df['IRT_State'].map(irt_map)
    df['EI_encoded'] = df['EI_State'].map(ei_map)
    df['GEO_encoded'] = df['GEO_State'].map(geo_map)
    df['UE_encoded'] = df['UE_State'].map(ue_map)
    df['INF_encoded'] = df['INF_State'].map(inf_map)
    df['GDP_encoded'] = df['GDP_State'].map(gdp_map)
    df['CP_encoded'] = df['CP_State'].map(cp_map)
    df['SP_encoded'] = df['Chosen_SP_State'].map(sp_map)

    # Split the data into features (X) and target (y)
    X = df[['IR_encoded', 'IRT_encoded', 'EI_encoded', 'GEO_encoded', 'UE_encoded', 'INF_encoded', 'GDP_encoded', 'CP_encoded']]
    y = df['SP_encoded']

    # Concatenate X and y to form the training set for learning the BN structure
    df_train = pd.concat([X, y], axis=1)

    # Define prior edges (dependency structure)
    priors = [
        ('IR_encoded', 'SP_encoded'),
        ('IRT_encoded', 'IR_encoded'),
        ('EI_encoded', 'SP_encoded'),
        ('UE_encoded', 'SP_encoded'),
        ('GEO_encoded', 'UE_encoded'),
        ('INF_encoded', 'GDP_encoded'),
        ('GDP_encoded', 'CP_encoded'),
        ('CP_encoded', 'SP_encoded')
    ]

    # Perform Hill Climb Search with priors
    hc = HillClimbSearch(df_train)
    best_dag = hc.estimate(scoring_method=BicScore(df_train), fixed_edges=priors)

    # Initialize BayesianNetwork with learned structure
    best_model = BayesianNetwork(best_dag.edges())
    print("Learned structure (edges):", best_model.edges())

    # Fit parameters using BayesianEstimator
    try:
        best_model.fit(df_train, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=5)
        print("Model fitted successfully.")
    except Exception as e:
        print("Error during fitting:", str(e))
        continue

    # Predict on training data and calculate K-L divergence
    try:
        predictions = best_model.predict(X)
        predicted_probabilities = predictions['SP_encoded']

        ground_truth_probabilities = y.value_counts(normalize=True).sort_index()
        predicted_probabilities = predicted_probabilities.value_counts(normalize=True).sort_index()

        # Reindex both distributions and add smoothing
        all_categories = sorted(set(ground_truth_probabilities.index).union(set(predicted_probabilities.index)))
        ground_truth_probabilities = ground_truth_probabilities.reindex(all_categories, fill_value=epsilon)
        predicted_probabilities = predicted_probabilities.reindex(all_categories, fill_value=epsilon)

        kl_divergence = entropy(pk=ground_truth_probabilities, qk=predicted_probabilities)
        std_dev = np.std(predicted_probabilities - ground_truth_probabilities)

        results.append({
            'Sample_Size': sample_size,
            'K-L_Divergence': kl_divergence,
            'Standard_Deviation': std_dev
        })

        print(f"K-L Divergence: {kl_divergence:.4f}")
        print(f"Standard Deviation: {std_dev:.4f}")

    except Exception as e:
        print("Error during prediction:", str(e))

# Save the K-L divergence results
results_df = pd.DataFrame(results)
results_df.to_csv('kl_std_bic_with_expanded_nodes.csv', index=False)

print("\nAll results have been saved.")


Processing sample size: 50


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('IRT_encoded', 'IR_encoded'), ('EI_encoded', 'SP_encoded'), ('GEO_encoded', 'UE_encoded'), ('UE_encoded', 'SP_encoded'), ('INF_encoded', 'GDP_encoded'), ('GDP_encoded', 'CP_encoded'), ('CP_encoded', 'SP_encoded')]
Model fitted successfully.


  0%|          | 0/50 [00:00<?, ?it/s]

K-L Divergence: 0.0011
Standard Deviation: 0.0163

Processing sample size: 100


...

K-L Divergence: 0.0145
Standard Deviation: 0.0586

Processing sample size: 20000


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('IRT_encoded', 'IR_encoded'), ('EI_encoded', 'SP_encoded'), ('EI_encoded', 'IR_encoded'), ('EI_encoded', 'IRT_encoded'), ('GEO_encoded', 'UE_encoded'), ('UE_encoded', 'SP_encoded'), ('INF_encoded', 'GDP_encoded'), ('GDP_encoded', 'CP_encoded'), ('CP_encoded', 'SP_encoded')]
Model fitted successfully.


  0%|          | 0/3975 [00:00<?, ?it/s]

K-L Divergence: 0.0044
Standard Deviation: 0.0312

All results have been saved.
